In [15]:
import time # 실행문 딜레이
import re # 정규표현식 모듈
import pandas as pd # 데이터프레임을 만들기 위한 모듈
from selenium import webdriver #크롤링 하기 위한 모듈
from selenium.webdriver import ActionChains # 마우스 커서로 가리킨 다음 클릭하는 모듈
from selenium.webdriver.common.by import By # XPATH 등으로 검색하기 위한 모듈
from selenium.webdriver.support.ui import WebDriverWait # 검색해야하는 엘리먼트를 찾을 때 까지 기다리는 모듈
from selenium.webdriver.support import expected_conditions as EC # 버튼 등이 이벤트 처리할 수 있는지를 확인하는 모듈
from bs4 import BeautifulSoup

In [18]:
def fetch_starbucks():
    starbucks_url = 'https://www.starbucks.co.kr/index.do'
    driver = webdriver.Chrome() # 크롬 driver 객체
    driver.maximize_window() # 브라우저 창 최대화
    driver.get(starbucks_url)
    time.sleep(1)

    # ActionChains(): 마우스나 키보드와 같은 복잡한 사용자 상호작용을 시뮬레이션하는 데 사용
    action = ActionChains(driver)
    first_tag = driver.find_element(By.CSS_SELECTOR, '#gnb > div > nav > div > ul > li.gnb_nav03')
    second_tag = driver.find_element(By.CSS_SELECTOR, '#gnb > div > nav > div > ul > li.gnb_nav03 > div > div > div > ul:nth-child(1) > li:nth-child(3) > a')
    action.move_to_element(first_tag).move_to_element(second_tag).click().perform() # ActionChains를 이용하여 첫번째 태그와 두번째 태그를 찾아서 클릭을 실행
    time.sleep(1)
    # #container > div > form > fieldset > div > section > article.find_store_cont > article > article:nth-child(4) > div.loca_step1 > div.loca_step1_cont > ul > li:nth-child(1) > a
    seoul_tag = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.CSS_SELECTOR, '#container > div > form > fieldset > div > section > article.find_store_cont > article > article:nth-child(4) > div.loca_step1 > div.loca_step1_cont > ul > li:nth-child(1) > a'))
    )
    seoul_tag.click()
    # time.sleep(5)
    
    # 매장 정보를 담기 위한 빈 리스트
    store_list = []
    addr_list = []
    lat_list = []
    lng_list = []

    # set_gugun_cd_btn
    # #mCSB_2_container > ul > li:nth-child(1) > a
    WebDriverWait(driver, 5).until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'set_gugun_cd_btn'))) # set_gugun_cd_btn으로 된 모든 엘리먼트가 떠있는지 확인
    gu_elements = driver.find_elements(By.CLASS_NAME, 'set_gugun_cd_btn')
    gu_elements[0].click()
    # time.sleep(5)

    # quickSearchResultBoxSidoGugun
    WebDriverWait(driver, 5).until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'quickResultLstCon')))
    req = driver.page_source # parsing
    soup = BeautifulSoup(req)
    stores = soup.find('ul', 'quickSearchResultBoxSidoGugun').find_all('li')

    for store in stores:
        store_name = store.find('strong').text
        store_addr = store.find('p').text
        store_addr = re.sub(r'\d{4}-\d{4}$', '', store_addr).strip() # ****-**** 형식은 제거
        store_lat = store['data-lat'] #태그 안에 속성 찾을 때
        store_lng = store['data-long']
        store_list.append(store_name)
        addr_list.append(store_addr)
        lat_list.append(store_lat)
        lng_list.append(store_lng)

    df = pd.DataFrame({
        'store': store_list,
        'addr': addr_list,
        'lat': lat_list,
        'lng': lng_list
    })

    driver.quit()
    return df
        

In [19]:
starbucks_df = fetch_starbucks()
starbucks_df

,store,addr,lat,lng
0,역삼아레나빌딩,서울특별시 강남구 언주로 425 (역삼동),37.501087,127.043069
1,논현역사거리,서울특별시 강남구 강남대로 538 (논현동),37.510178,127.022223
2,신사역성일빌딩,서울특별시 강남구 강남대로 584 (논현동),37.5139309,127.0206057
3,국기원사거리,서울특별시 강남구 테헤란로 125 (역삼동),37.499517,127.031495
4,대치재경빌딩,서울특별시 강남구 남부순환로 2947 (대치동),37.494668,127.062583
...,...,...,...,...
628,상봉역,서울특별시 중랑구 망우로 307 (상봉동),37.59689,127.08647
629,묵동,"서울특별시 중랑구 동일로 952 (묵동, 로프트원 태릉입구역) 1층",37.615368,127.076633
630,망우동,서울특별시 중랑구 망우로 407 (망우동),37.5995083771241,127.097538751682
631,양원역,서울특별시 중랑구 양원역로10길 3 (망우동),37.6066536267232,127.106359790053


In [20]:
starbucks_df.to_csv('starbucks_seoul2.csv', index=False, encoding='utf-8-sig')
print('데이터가 starbucks_seoul2.csv 파일로 저장되었습니다.')

데이터가 starbucks_seoul2.csv 파일로 저장되었습니다.
